In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install sentence-transformers
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
from PyPDF2 import PdfReader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.callbacks import get_openai_callback

In [3]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# location of the pdf file/files.
reader = PdfReader('/content/drive/My Drive/sample_doc_1.pdf')

In [6]:
reader

In [7]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [8]:
# raw_text

In [10]:
raw_text[100:1000]

'd.edu\nAbstract\nThis project explores and compares different NLP architectures performance on The\nStanford Question Answering Data(SQuAD). The basest performances model is\nBERT based model with addition layer on original BERT SQuAD implementation.\nThis model’s F1 score and Exact Match score is 73.77. But this model takes long\ntime to train each epoch, and does not improve after trained 3 epochs. QANet\nbased model provided slightly lower F1 and EM score, but continuous improving\nwith training.\n1 Introduction\nRecent development in Deep Learning in Nature Language Processing (NLP) has reach better than\nhuman performance on multiple NLP tasks. Recurrent neural networks(RNN), long short-term\nmemory(LSTM)[ 2] are widely use NLP model. The sequence nature of RNN and LSTM limited the\nneural network model to parallel calculation, and requires long training time for large dataset. This\nleads to app'

In [19]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex=False,

)
texts = text_splitter.split_text(raw_text)

In [20]:
len(texts)

24

In [21]:
texts[10]

'This report use CS224N default ﬁnal project’s data split of SQuAD dataset. Each example in training\nand development data sets has (context, question, answer) triples, and each example in testing data\nset has (context, question) triples. The model take context and question as input, and try to output the\nstart and end position of the answer.\nThe train, development, and test data set have 129,941, 6078, and 5915 examples, respectfully. One\ncontext has multiple questions related to it; and one question has only one answer in the train dataset,\nbut might have slightly different answers in development dataset.\nContext : Following the disbandment of Destiny’s Child in June 2005, she released her second\nsolo album, B’Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar".\nBeyoncé also ventured into acting , with a Golden Globe-nominated performance in Dreamgirls\n(2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to'

In [17]:
texts[1]

'leads to application of Convolution Neural Networks(CNN) and Attention layer that enable faster\ntraining and smaller model with similar or higher preference. In 2018, the application of Bidirectional\nEncoder Representations from Transformers(BERT) has reached new state of art in various tasks,\nincluding reading comprehension, and encourage a multitask model structure.\nReading comprehension or question answering task has gained great popularity in NLP task due to\nits wide application. Stanford Question Answering Dataset (SQuAD) is one of most widely used\nreading comprehension dataset and base of data used in this paper. The ofﬁcial SQuAD 2.0 dataset\nconsists of 100,000 questions posed by crowd-workers on a set of Wikipedia articles. About half of\nquestions are answerable, and the answer to answerable every question is a segment of text, or span,\nfrom the corresponding reading context.\nThis paper focused on comparing architectures with different complexities level on performan

In [ ]:
# # Download embeddings from OpenAI
# embeddings = OpenAIEmbeddings()

In [22]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
# stores embeddings in the FAISS vector store
docsearch = FAISS.from_texts(texts, embeddings)

In [24]:
docsearch

In [63]:
# basic retriever used
retriever = docsearch.as_retriever()

In [64]:
docs = retriever.get_relevant_documents("What was the best model performance achieved?")

In [65]:
print(docs)

[Document(page_content='Largest pretrained contextual embedding model, a BERT based model with additional layers for\nSQuAD, provides best performance among the models that have been tested., F1 score of 73.77 and\n72.95 on development and test sets. Median size model QANet provides second best and BiDAF\nbased have lowest performances. But BERT take very long time to train for each epoch, and reach\n7performance cap within 3 epoch. On the other hand, Non-PCE models have lower initial performance,\nbut can improve with more train epochs.\nUsing pretrained character level embedding, regularization improves model performance slightly,\nbut not a signiﬁcant amount. Model size and the depth are the determination factors of model\nperformance.\nBest model requires a long training time and large GPU RAM. This could be a bottle neck when\napply the-state-of-art model in industry.\nAcknowledgement\nThanks Professor Chris Manning as well as CS224N teaching assistant team who provided this'), Do

In [66]:
from langchain.prompts import ChatPromptTemplate
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))]


In [67]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What was the best model performance achieved?"
rag_chain.invoke(query)

'The best model performance achieved was by the BERT based model with additional layers for SQuAD, which had an F1 score of 73.77 on the development set and 72.95 on the test set. This model outperformed other models tested, with the median size model QANet coming in second. The performance of the models was determined by factors such as model size, depth, and training time.'

======================================================================================================

## 2nd Method

In [30]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [31]:
# uses the load_qa_chain
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [73]:
# get top 2 similar chunks
query = "What was the best model performance achieved?"
docs = docsearch.similarity_search(query, k=2)

In [74]:
print(docs)

[Document(page_content='Largest pretrained contextual embedding model, a BERT based model with additional layers for\nSQuAD, provides best performance among the models that have been tested., F1 score of 73.77 and\n72.95 on development and test sets. Median size model QANet provides second best and BiDAF\nbased have lowest performances. But BERT take very long time to train for each epoch, and reach\n7performance cap within 3 epoch. On the other hand, Non-PCE models have lower initial performance,\nbut can improve with more train epochs.\nUsing pretrained character level embedding, regularization improves model performance slightly,\nbut not a signiﬁcant amount. Model size and the depth are the determination factors of model\nperformance.\nBest model requires a long training time and large GPU RAM. This could be a bottle neck when\napply the-state-of-art model in industry.\nAcknowledgement\nThanks Professor Chris Manning as well as CS224N teaching assistant team who provided this'), Do

In [75]:
# check token usage
with get_openai_callback() as cb:
  response=chain.run(input_documents=docs, question=query)
  print(cb)



Tokens Used: 511
	Prompt Tokens: 459
	Completion Tokens: 52
Successful Requests: 1
Total Cost (USD): $0.0007925000000000001


In [76]:
print(response)


 The best model performance was achieved by the largest pretrained contextual embedding model, a BERT based model with additional layers for SQuAD, which had an F1 score of 73.77 on the development dataset and 72.95 on the test dataset.
